In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical

2024-08-04 00:45:43.021679: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-04 00:45:43.156370: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-04 00:45:44.392716: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# Carrega a base de dados a partir de seu caminho
data = np.load('dataset.npz', allow_pickle=True)
X_test = data['X_test']
X_train = data['X_train']
y_test = data['y_test']
y_train = data['y_train']

In [4]:
# Convert labels to categorical
num_classes = len(np.unique(y_train))
y_train_categorical = to_categorical(y_train, num_classes)

In [5]:
num_classes

5

In [6]:
# Scale
scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

In [7]:
#Model
initializer = keras.initializers.GlorotNormal(seed=12227)

inputs_layer = keras.layers.Input(shape=(X_test_sc.shape[1],))

H = keras.layers.Dense(units=128, activation='relu')(inputs_layer)
H = keras.layers.Dense(units=64, activation='relu')(H)
H = keras.layers.Dense(units=32, activation='relu')(H)
H = keras.layers.Dense(units=32, activation='relu')(H)
outputs_layer = keras.layers.Dense(units=num_classes, activation='softmax')(H)

model = keras.models.Model(inputs=inputs_layer, outputs=outputs_layer)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
history = model.fit(X_train_sc, y_train_categorical, epochs=100, batch_size=32)

Epoch 1/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.4734 - loss: 2.5485
Epoch 2/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.7968 - loss: 1.0792
Epoch 3/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.9060 - loss: 0.3516
Epoch 4/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.9288 - loss: 0.1897
Epoch 5/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.9618 - loss: 0.1395
Epoch 6/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.9811 - loss: 0.1001
Epoch 7/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.9635 - loss: 0.2195
Epoch 8/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.9829 - loss: 0.0523
Epoch 9/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.9645 - loss: 0.2045
Epoch 10/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.9475 - loss: 0.3496
Epoch 11/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.9124 - loss: 0.5142
Epoch 12/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step

In [9]:
y_pred_prob = model.predict(X_test_sc)

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


In [10]:
y_pred_classes = np.argmax(y_pred_prob, axis=1)
# Gera um DataFrame no formato esperado da submissão
num_samples = X_test.shape[0]
submission_df = pd.DataFrame({
    'ID': np.arange(1, num_samples + 1),
    'Prediction': y_pred_classes
})

# Salva o arquivo CSV no diretório atual
submission_df.to_csv('submission_task2_v2.csv', index=False)